# 다중 선형 회귀(Multivariable Linear Regression)


x가 1개인 선형 회귀를 **단순 선형 회귀(Simple Linear Regression)**라고 했다.

y를 예측하는 x가 다수일 때 **다중 선형 회귀(Multivariable Linear Regression)**라고 한다.

## 사용 예제

|**Quiz1**|**Quiz2**|**Quiz3**|**Final(y)**|
|---|---|---|---|
|73|80|75|152|
|93|88|93|185|
|89|91|80|180|
|96|98|100|196|
|73|66|70|142|

다음과 같이 3개의 퀴즈 점수로 최종 점수를 예측하는 모델을 만들어 볼 계획이다.


### $$H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b$$
**<center>독립 변수 x의 개수가 3개인 수식<center>**

## Pytorch로 직접 구현하기

In [1]:
import torch
import torch.nn as nn # 선형 회귀 불러오기 위해 사용
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# 훈련 데이터
x1_train = torch.FloatTensor([[73],[93],[89],[86],[73]])
x2_train = torch.FloatTensor([[80],[88],[91],[98],[66]])
x3_train = torch.FloatTensor([[75],[93],[80],[100],[70]])
y_train = torch.FloatTensor([[152],[185],[180],[196],[142]])

In [3]:
# 가중치 w와 편향 b 설정, requires_grad = 기울기를 저장
w1 = torch.zeros(1, requires_grad = True)
w2 = torch.zeros(1, requires_grad = True)
w3 = torch.zeros(1, requires_grad = True)
b = torch.zeros(1, requires_grad = True)

In [4]:
# optimizer 설정, 학습률 0.00001
optimizer = optim.SGD([w1,w2,w3, b], lr = 1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1) :
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} w1: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs,w1.item(),w2.item(),w3.item(),b.item(),cost.item()))

C:\Users\admin\anaconda3\lib\site-packages\torch\autograd\__init__.py:147: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:115.)
  Variable._execution_engine.run_backward(


Epoch    0/1000 w1: 0.286 w3: 0.294 b: 0.290 Cost: 0.003420
Epoch  100/1000 w1: 0.670 w3: 0.695 b: 0.680 Cost: 0.007792
Epoch  200/1000 w1: 0.667 w3: 0.700 b: 0.678 Cost: 0.007542
Epoch  300/1000 w1: 0.665 w3: 0.705 b: 0.676 Cost: 0.007294
Epoch  400/1000 w1: 0.662 w3: 0.709 b: 0.673 Cost: 0.007048
Epoch  500/1000 w1: 0.660 w3: 0.713 b: 0.671 Cost: 0.006803
Epoch  600/1000 w1: 0.658 w3: 0.718 b: 0.669 Cost: 0.006559
Epoch  700/1000 w1: 0.656 w3: 0.721 b: 0.667 Cost: 0.006316
Epoch  800/1000 w1: 0.654 w3: 0.725 b: 0.666 Cost: 0.006075
Epoch  900/1000 w1: 0.652 w3: 0.729 b: 0.664 Cost: 0.005834
Epoch 1000/1000 w1: 0.650 w3: 0.732 b: 0.662 Cost: 0.005595


## nn.module을 사용하여 구현하기

In [5]:
# 데이터 
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [6]:
# 모델을 선언 및 초기화, 다중 선형 회귀 이므로 input_dim = 3, output_dim = 1
model = nn.Linear(3,1)

3개의 입력 x에 대해 하나의 출력 y를 가지므로, torch.nn.Linear의 인자로 3,1을 사용했다. 

In [7]:
# 3개의 w값과 b 가져오기
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.5101, -0.5261,  0.4455]], requires_grad=True), Parameter containing:
tensor([0.4149], requires_grad=True)]


처음 출력된 3개의 값이 w, 두번째 출력되는 것이 b의 값이다. 두 값 모두 현재는 랜덤 초기화가 되어져 있으며, 두 출력 결과 모두 학습의 대상이므로 **requires_grad = True**가 되어져 있는 것을 볼 수 있다.

In [8]:
# 옵티마이저 설정
optimizer = optim.SGD(model.parameters(), lr = 1e-5)

nb_epochs = 2000

for epoch in range(nb_epochs + 1) :
    
    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)와 동일함 
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 MSE
    
    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # w와 b를 업데이트
    optimizer.step()
    
    if epoch % 100 == 0 :
        # 100번마다 로그 출력
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch,nb_epochs, cost.item()))

Epoch    0/2000 Cost: 18138.427734
Epoch  100/2000 Cost: 3.483390
Epoch  200/2000 Cost: 3.325043
Epoch  300/2000 Cost: 3.174954
Epoch  400/2000 Cost: 3.032701
Epoch  500/2000 Cost: 2.897868
Epoch  600/2000 Cost: 2.770078
Epoch  700/2000 Cost: 2.648943
Epoch  800/2000 Cost: 2.534109
Epoch  900/2000 Cost: 2.425257
Epoch 1000/2000 Cost: 2.322070
Epoch 1100/2000 Cost: 2.224263
Epoch 1200/2000 Cost: 2.131519
Epoch 1300/2000 Cost: 2.043565
Epoch 1400/2000 Cost: 1.960186
Epoch 1500/2000 Cost: 1.881137
Epoch 1600/2000 Cost: 1.806187
Epoch 1700/2000 Cost: 1.735089
Epoch 1800/2000 Cost: 1.667685
Epoch 1900/2000 Cost: 1.603754
Epoch 2000/2000 Cost: 1.543105


In [9]:
# 새로운 임의 데이터 생성
new_var = torch.FloatTensor([[73,80,75]])
# 임의 데이터 예측
pred_y = model(new_var)
print(pred_y)

tensor([[149.5582]], grad_fn=<AddmmBackward>)


In [10]:
# 학습 후의 w와 b의 값을 출력
print(list(model.parameters()))

[Parameter containing:
tensor([[0.9223, 0.1582, 0.9221]], requires_grad=True), Parameter containing:
tensor([0.4231], requires_grad=True)]


## 모델을 클래스로 구현하기

In [11]:
# 다시 데이터 불러오기
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [12]:
# 다중회귀 클래스 생성
class MultivariateLinearRegressionModel(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.linear = nn.Linear(3,1) # 다중 선형회귀이므로 input_dim = 3, output_dim = 1
        
    def forward(self, x) :
        return self.linear(x)

In [13]:
# model에 저장
model = MultivariateLinearRegressionModel()

In [14]:
# 옵티마이저 생성
optimizer = optim.SGD(model.parameters(), lr = 1e-5)

In [15]:
nb_epochs = 2000

for epoch in range(nb_epochs + 1) :
    # H(x) 계산
    prediction = model(x_train)
    # model(x_train)은 model.forward(x_train)와 동일함
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 MSE 함수
    
    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # w와 b를 업데이트
    optimizer.step()
    
    if epoch % 100 == 0 :
    # 100번마다 로그 출력
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
              epoch, nb_epochs, cost.item()))

Epoch    0/2000 Cost: 21379.408203
Epoch  100/2000 Cost: 1.050733
Epoch  200/2000 Cost: 1.007648
Epoch  300/2000 Cost: 0.966829
Epoch  400/2000 Cost: 0.928151
Epoch  500/2000 Cost: 0.891486
Epoch  600/2000 Cost: 0.856742
Epoch  700/2000 Cost: 0.823806
Epoch  800/2000 Cost: 0.792593
Epoch  900/2000 Cost: 0.763001
Epoch 1000/2000 Cost: 0.734959
Epoch 1100/2000 Cost: 0.708375
Epoch 1200/2000 Cost: 0.683164
Epoch 1300/2000 Cost: 0.659287
Epoch 1400/2000 Cost: 0.636633
Epoch 1500/2000 Cost: 0.615163
Epoch 1600/2000 Cost: 0.594798
Epoch 1700/2000 Cost: 0.575500
Epoch 1800/2000 Cost: 0.557194
Epoch 1900/2000 Cost: 0.539848
Epoch 2000/2000 Cost: 0.523391
